In [ ]:
!pip install transformers -qq
!pip install sentencepiece -qq
!pip install --user annoy -qq
!pip install -U sentence-transformers -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
len(embeddings[0])

768

In [ ]:
len(last_hidden_state[0])

768

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast
from sentence_transformers import SentenceTransformer

df = pd.read_csv("/content/data.csv", index_col=0)
model = SentenceTransformer('sentence-transformers/sentence-t5-large')

vectors = []
for index, row in tqdm(df.iterrows()):
    #print(row['description'], row['tags'])
    tags = ", ".join(ast.literal_eval(row['tags']))


    text = f"{row['description']} tags:{tags}, other params: market Cap:{row['marketCap']}, Last volume:{row['lastVolume']}, open:{row['open']}, previousClose:{row['previousClose']}"
    embeddings = model.encode([text])



    vectors.append(np.array(embeddings[0]))




69it [01:53,  1.64s/it]


In [ ]:
from annoy import AnnoyIndex
import random

f = 768

t = AnnoyIndex(f, 'angular')
for i,v in enumerate(vectors):
    t.add_item(i, v)

t.build(10)
t.save('model.ann')



u = AnnoyIndex(f, 'angular')
u.load('model.ann')

print([i for i in u.get_nns_by_item(0, 10) if i not in [4]][:5])

[0, 38, 64, 5, 41]


In [ ]:
[0, 38, 64, 41, 5]


4